# WuTong V9 Kaggle 一键训练

V9 核心改进（基于 V7 99.35%）：
- 混淆对专家模型：针对 top-5 混淆对训练专门的二分类器
- 置信度路由：主模型低置信样本交给专家判断
- 对抗验证：自动检测训练/测试分布偏移并调整权重
- 概率校准：CalibratedClassifierCV isotonic 校准
- +5 个精准区分特征（保守策略，不贪多）

直接 **Run All** 即可。预计 30-40 分钟。


In [ ]:
# 1) 克隆/更新代码
from pathlib import Path
import os

repo_dir = Path('/kaggle/working/WuTong')
if repo_dir.exists():
    print('Repo exists, pulling latest...')
    os.system(f'cd {repo_dir} && git pull')
else:
    print('Cloning repo...')
    os.system('git clone https://github.com/alltobebetter/WuTong.git /kaggle/working/WuTong')

%cd /kaggle/working/WuTong
!git log --oneline -n 5


In [ ]:
# 2) 安装依赖
!pip -q install -r requirements.txt
!pip -q install imbalanced-learn optuna

import torch
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))


In [ ]:
# 3) 准备数据
from pathlib import Path
import shutil

raw_dir = Path('data/raw')
raw_dir.mkdir(parents=True, exist_ok=True)

repo_data = list(Path('data_raw').glob('*.xlsx')) + list(Path('data_raw').glob('*.csv'))
if repo_data:
    print(f'从仓库 data_raw/ 找到 {len(repo_data)} 个数据文件')
    for src in repo_data:
        dst = raw_dir / src.name
        if not dst.exists():
            shutil.copy2(src, dst)
            print(f'  复制: {src.name}')
else:
    candidates = list(Path('/kaggle/input').rglob('*.xlsx')) + list(Path('/kaggle/input').rglob('*.csv'))
    print(f'从 /kaggle/input 找到 {len(candidates)} 个候选文件')
    for src in candidates[:20]:
        dst = raw_dir / src.name
        if not dst.exists():
            shutil.copy2(src, dst)

print('\ndata/raw files:')
for p in sorted(raw_dir.glob('*')):
    print(f'  - {p.name} ({p.stat().st_size / 1024:.0f} KB)')


In [ ]:
# 4) 预处理 + 增强 + 外部数据
import glob, os

source_files = glob.glob('data/raw/*.xlsx') + glob.glob('data/raw/*.csv')
if not source_files:
    raise RuntimeError('data/raw 没有可用数据')

src = source_files[0]
print('Using source file:', src)
ret = os.system(f'python -u scripts/ingest.py "{src}"')
if ret != 0: raise RuntimeError('ingest 失败')

ret = os.system('python -u scripts/augment_data.py --target-size 30000 --ratio 2.5')
if ret != 0: raise RuntimeError('augment 失败')

ret = os.system('python -u scripts/integrate_csic2010.py')
if ret != 0:
    print('⚠️ 外部数据集下载失败，将仅使用原始+增强数据训练')

print('preprocess pipeline done')


In [ ]:
# 5) V9 训练
import subprocess, sys

print('='*80)
print('🚀 V9 训练开始')
print('='*80)
print('\n核心改进（vs V7 99.35%）:')
print('  - 混淆对专家 + 置信度路由')
print('  - 对抗验证 + 样本权重调整')
print('  - 概率校准 CalibratedClassifierCV')
print('  - +5 个精准区分特征\n')

cmd = [
    sys.executable, '-u', 'scripts/train_v9.py',
    '--version', 'v9.0.0-kaggle',
    '--cv-splits', '10',
    '--optuna-trials', '30',
    '--confidence-threshold', '0.7',
]

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in proc.stdout:
    print(line, end='')

ret = proc.wait()
if ret != 0:
    raise RuntimeError(f'训练失败，退出码: {ret}')

print('\n' + '='*80)
print('✅ V9 训练完成！')
print('='*80)


In [ ]:
# 6) 查看训练结果
from pathlib import Path
import json

manifest_path = Path('models/v9.0.0-kaggle/manifest.json')

if manifest_path.exists():
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest = json.load(f)

    print('='*80)
    print('📊 V9 训练结果')
    print('='*80)
    print(f"\n版本: {manifest['version']}")
    print(f"训练时间: {manifest['trained_at']}")
    print(f"数据量: {manifest['data_rows']} 条")
    print(f"特征数: {len(manifest['feature_list'])} 个")

    print('\n' + '='*80)
    print('🎯 模型性能')
    print('='*80)

    metrics = manifest['metrics']
    for name in ['xgboost', 'catboost', 'lightgbm', 'extratrees']:
        if name not in metrics: continue
        m = metrics[name]
        cv_str = f", CV: {m['cv_accuracy']:.4f}" if m.get('cv_accuracy') else ''
        print(f"  {name}: Acc={m['test_accuracy']:.4f}, F1={m['test_f1']:.4f}{cv_str}")

    e = metrics['ensemble']
    print(f"\n🏆 最终模型（{e['ensemble_type']}）")
    print(f"  准确率: {e['test_accuracy']:.4f} ({e['test_accuracy']*100:.2f}%)")
    print(f"  F1 分数: {e['test_f1']:.4f}")

    if 'all_candidates' in metrics:
        print('\n  所有候选:')
        for name, acc in metrics['all_candidates'].items():
            print(f'    {name}: {acc:.4f}')

    adv = manifest.get('adversarial_validation', {})
    if adv:
        print(f"\n  对抗验证 AUC: {adv.get('auc', 'N/A')}")
        print(f"  分布偏移: {'是' if adv.get('distribution_shift') else '否'}")

    v7_acc = 0.9935
    delta = e['test_accuracy'] - v7_acc
    print(f"\n  vs V7: {'+' if delta >= 0 else ''}{delta*100:.2f}% ({'↑' if delta > 0 else '↓'})")
else:
    print('❌ 未找到训练结果文件！')


In [ ]:
# 7) 打包下载
from pathlib import Path
import shutil

model_dir = Path('models/v9.0.0-kaggle')
if model_dir.exists():
    print('📦 打包模型文件...')
    archive = shutil.make_archive(
        '/kaggle/working/models_v9.0.0-kaggle', 'zip',
        'models', 'v9.0.0-kaggle'
    )
    print(f'✅ 打包完成: {archive}')
    print(f'\n文件大小: {Path(archive).stat().st_size / 1024 / 1024:.2f} MB')
    print('\n💡 在 Kaggle 右侧 Output 面板下载')
else:
    print('❌ 模型目录不存在！')


## V9 vs V7 改进说明

| 维度 | V7 (99.35%) | V9 |
|------|-------------|-----|
| 特征数 | 49 | 54 (+5 精准区分特征) |
| 混淆对专家 | 无 | 5 个二分类专家模型 |
| 置信度路由 | 无 | 低置信样本交给专家 |
| 对抗验证 | 无 | 自动检测分布偏移 |
| 概率校准 | 无 | isotonic 校准 |
| Stacking | passthrough=False | passthrough=False (保持) |

V9 的核心思路：不做大改，只在 V7 的 38 条错误上做精准手术。
